<center style="font-size:48px;">Clean Up</center>

In [286]:
import pandas as pd 

cars = pd.read_csv('../Data/car-assignments.csv')
cc = pd.read_csv('../Data/cc_data.csv', encoding='cp1252', parse_dates=['timestamp'])
gps = pd.read_csv('../Data/gps.csv', parse_dates=['Timestamp'])
loyalty = pd.read_csv('../Data/loyalty_data.csv', encoding='cp1252', parse_dates=['timestamp'])

In [287]:
locations = gps.merge(cars, left_on='id', right_on='CarID', how='left')
locations.drop(columns='CarID', inplace =True)
locations.head()

,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle
0,2014-01-06 06:28:01,35,36.076225,24.874689,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
1,2014-01-06 06:28:01,35,36.076220,24.874596,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor


In [288]:
print("People with first name Driver : {}".format(sum(sum([locations.FirstName == "Truck"]))))
print("People with last name Driver : {}".format(sum(sum([locations.LastName == "Driver"]))))

People with first name Driver : 0
People with last name Driver : 0


In [289]:
locations['CurrentEmploymentType'] = locations['CurrentEmploymentType'].fillna('Facilities')
locations['CurrentEmploymentTitle'] = locations['CurrentEmploymentTitle'].fillna('Truck Driver')
locations['LastName'] = locations['LastName'].fillna('Driver')
locations['FirstName'] = locations['FirstName'].fillna('Truck')
locations['LastName'] = locations.apply(lambda x: 'Driver_{}'.format(x['id']) if x['LastName'] == 'Driver' else x['LastName'], axis =1 )

In [290]:
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.second)

In [291]:
locations[['lat', 'long']] = locations.groupby(['day', 'hour', 'minute', 'second', 'FirstName', 'LastName'])['lat', 'long'].transform('mean')
locations.drop_duplicates()

/tmp/ipykernel_42806/470201690.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  locations[['lat', 'long']] = locations.groupby(['day', 'hour', 'minute', 'second', 'FirstName', 'LastName'])['lat', 'long'].transform('mean')


,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second
0,2014-01-06 06:28:01,35,36.076223,24.874643,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,1
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,3
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,5
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,6
5,2014-01-06 06:28:07,35,36.076191,24.874056,Vasco-Pais,Willem,Executive,Environmental Safety Advisor,6,6,28,7
...,...,...,...,...,...,...,...,...,...,...,...,...
685164,2014-01-19 20:56:43,30,36.058110,24.902130,Resumir,Felix,Security,Security Group Manager,19,20,56,43
685165,2014-01-19 20:56:47,30,36.058258,24.901774,Resumir,Felix,Security,Security Group Manager,19,20,56,47
685166,2014-01-19 20:56:48,30,36.058296,24.901711,Resumir,Felix,Security,Security Group Manager,19,20,56,48
685167,2014-01-19 20:56:49,30,36.058304,24.901620,Resumir,Felix,Security,Security Group Manager,19,20,56,49


In [292]:
cc['Is_Loyalty'] = False
loyalty['Is_Loyalty'] = True

In [293]:
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        cc[unit] = cc['timestamp'].apply(lambda x: x.day)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        cc[unit] = cc['timestamp'].apply(lambda x: x.hour)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        cc[unit] = cc['timestamp'].apply(lambda x: x.minute)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        cc[unit] = cc['timestamp'].apply(lambda x: x.second)
        loyalty[unit] = loyalty['timestamp'].apply(lambda x: x.second)

cc['cents'] = round(cc.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))
loyalty['cents'] = round(loyalty.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))

In [294]:
for index, row in cc.iterrows():
    first = row['FirstName']
    last = row['LastName']
    location = row['location']
    day = row['day']
    cents = row['cents']
    temp = loyalty[loyalty.FirstName == first]
    temp = temp[temp.LastName == last]
    temp = temp[temp.location == location]
    temp = temp[temp.day == day]
    temp = temp[temp.cents == cents]
    if len(temp) >= 1:
        cc.loc[index, 'Is_Loyalty'] = True
        cc.loc[index, 'price'] = temp.price.values

buys = pd.concat([cc, loyalty]).drop_duplicates(['FirstName', 'LastName', 'location', 'day', 'cents'], keep='first')
buys = buys.merge(cars, left_on=['LastName', 'FirstName'], right_on=['LastName', 'FirstName'], how= 'left')


In [295]:
import numpy as np
buys.fillna({'CurrentEmploymentType' : 'Other', 'CurrentEmploymentTitle':'Other'}, inplace= True)
def CarNulls(x):
    if pd.isna(x['CarID']):
        if x['CurrentEmploymentTitle'] == 'Truck Driver':
            return 100
        else:
            return 0
    else:
        return x['CarID']

buys['CarID'] = buys.apply(lambda x: CarNulls(x), axis =1)


In [297]:
locations.to_csv('../CheckPoints/Locations_Clean.csv')
buys.to_csv('../CheckPoints/Buys_Clean.csv')

<div>
    <span  style="width:600px;display:inline-block;text-align:left">
        <a href="./FurtherEDA.ipynb">&#60;&#60;Further Exploratory Data Analysis</a>
    </span>
    <span style="width:600px;display:inline-block;text-align:right">
        <a href="./Modeling.ipynb">Modeling&#62;&#62;</a>
    </span>
</div>
<div>
    <center>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="./Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="../README.md">Table of Contents</a>
        </span>
    </center>
</div>